In [65]:
# Import the packages
import pandas as pd
from google.cloud import bigquery
from google.cloud import bigquery_storage
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import concurrent.futures
pd.set_option("display.max_columns", 100)
import warnings
warnings.filterwarnings(action="ignore")

In [17]:
# Declare some global inputs
entity_id = "PY_CL"
test_names = ["'CL_20221201_R_B0_O_ElasticityCLRestaurants'"]
test_start_date = "2022-12-01"
test_end_date = "2023-01-12"
scheme_ids = ["634", "635"]
num_bootstrap_samples = 1000 # Number of data points to have on the histogram
cl = 0.95
left_threshold = round((1 - cl) / 2, 4)
right_threshold = round((1 - (1 - cl) / 2), 4)

In [7]:
# Instantiate the BQ client
client = bigquery.Client(project="logistics-customer-staging")
bqstorage_client = bigquery_storage.BigQueryReadClient()

g:\My Drive\APAC\venv_dh\lib\site-packages\google\auth\_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


### Orders query

In [44]:
test_query = """
    WITH load_scheme_data AS (
        SELECT 
            entity_id,
            scheme_id,
            scheme_active_from,
            IFNULL(scheme_active_to, CURRENT_TIMESTAMP()) AS scheme_active_to,
            scheme_component_configs.travel_time_config
        FROM `fulfillment-dwh-production.cl.pricing_configuration_versions`
    ),
    
    -- Load order data and join PM data
    dps_order_raw as (
    SELECT
        -- Identifiers and supplementary fields     
        -- Date and time
        a.created_date AS created_date_utc,
        a.order_placed_at,

        -- Location of order
        ent.segment AS region,
        a.entity_id,
        a.country_code,
        a.city_name,
        a.city_id,
        a.zone_name,
        a.zone_id,

        -- Order/customer identifiers and session data
        a.variant,
        a.experiment_id AS test_id,
        b.test_name,
        a.platform_order_code,
        a.scheme_id,
        a.vendor_price_scheme_type,	-- The assignment type of the scheme to the vendor during the time of the order, such as "Automatic", "Manual", "Campaign", and "Country Fallback".
        
        -- Vendor data and information on the delivery
        a.vendor_id,
        b.target_group AS target_group_bi,
        a.is_in_treatment,
        a.chain_id,
        a.chain_name,
        a.vertical_type,
        a.delivery_status,
        a.is_own_delivery,
        a.exchange_rate,

        -- Business KPIs (These are the components of profit)
        a.dps_delivery_fee_local,
        a.dps_travel_time_fee_local,
        a.dps_surge_fee_local,
        a.delivery_fee_local,
        IF(a.is_delivery_fee_covered_by_discount = TRUE OR a.is_delivery_fee_covered_by_voucher = TRUE, 0, a.delivery_fee_local) AS delivery_fee_local_incl_disc_and_vouchers,
        CASE WHEN ent.segment != "MENA" THEN a.commission_local ELSE COALESCE(mn.commission_amount_lc, a.commission_local) END AS commission_local,
        a.joker_vendor_fee_local,
        COALESCE(a.service_fee_local, 0) AS service_fee_local,
        a.mov_customer_fee_local AS sof_local,
        a.delivery_costs_local,
        CASE
            WHEN ent.segment IN ("Europe", "Asia") THEN COALESCE( -- Get the delivery fee data of Pandora countries from Pandata tables
                pd.delivery_fee_local, 
                -- In 99 pct of cases, we won"t need to use that fallback logic as pd.delivery_fee_local is reliable
                IF(a.is_delivery_fee_covered_by_discount = TRUE OR a.is_delivery_fee_covered_by_voucher = TRUE, 0, a.delivery_fee_local)
            )
            -- If the order comes from a non-Pandora country, use delivery_fee_local
            WHEN ent.segment NOT IN ("Europe", "Asia") THEN (CASE WHEN a.is_delivery_fee_covered_by_discount = TRUE OR a.is_delivery_fee_covered_by_voucher = TRUE THEN 0 ELSE a.delivery_fee_local END)
        END AS actual_df_paid_by_customer,
        a.gfv_local,
        a.gmv_local,
        a.dps_travel_time,
        e.cluster entity_cluster,
        v.cluster area_cluster,
    FROM `fulfillment-dwh-production.cl.dps_sessions_mapped_to_orders_v2` a
    LEFT JOIN `fulfillment-dwh-production.cl.dps_ab_test_orders_v2` b ON a.entity_id = b.entity_id AND a.order_id = b.order_id
    LEFT JOIN `logistics-data-storage-staging.long_term_pricing.vendors_clustered_entity` e on a.entity_id = e.entity_id and a.vendor_id = e.vendor_id 
    LEFT JOIN `logistics-data-storage-staging.long_term_pricing.vendors_clustered` v on v.entity_id = a.entity_id and a.vendor_id = v.vendor_id
    LEFT JOIN `tlb-data-prod.data_platform.fct_billing` mn ON a.platform_order_code = CAST(mn.order_id AS STRING) AND a.entity_id IN ("TB_OM", "TB_IQ", "TB_AE", "TB_KW", "YS_TR", "TB_QA", "TB_JO", "HF_EG", "HS_SA", "TB_BH")
    LEFT JOIN `fulfillment-dwh-production.pandata_curated.pd_orders` pd ON TRUE -- Contains info on the orders in Pandora countries 
        AND a.entity_id = pd.global_entity_id
        AND a.platform_order_code = pd.code 
        AND a.created_date = pd.created_date_utc -- There is no country_code field in this table
    INNER JOIN `fulfillment-dwh-production.curated_data_shared_coredata.global_entities` ent ON a.entity_id = ent.global_entity_id -- Get the region associated with every entity_id
    WHERE TRUE
        AND a.created_date between DATE("{test_start_date}") and DATE("{test_end_date}")
        AND b.test_name IN ({test_names})
        AND (a.scheme_id IS NOT NULL AND scheme_id IN ({scheme_ids}))
        AND a.entity_id = "{entity_id}"
        AND a.is_in_treatment = TRUE -- Retrieve orders in the treatment scope only
        AND a.is_sent -- Successful orders
        AND a.is_own_delivery -- OD orders only
        AND a.variant != "Original" -- Exclude orders from ASAs
        AND a.is_match_experiment_vertical
        AND (e.cluster IS NOT NULL AND e.cluster NOT IN ('Insufficient data'))
        AND (v.cluster IS NOT NULL AND v.cluster NOT IN ('Insufficient data'))
    ),

    dps_order_with_pm AS (
        SELECT
            dps.*,
            sch.* EXCEPT (entity_id, scheme_id, scheme_active_from, scheme_active_to),
            (
                SELECT MIN(tier) + 1 AS tier
                FROM UNNEST(travel_time_config) tt
                WITH OFFSET AS tier 
                WHERE dps_travel_time <= IFNULL(tt.travel_time_threshold, 999999) 
            ) AS tt_tier
        FROM dps_order_raw dps
        LEFT JOIN load_scheme_data sch ON TRUE
            AND dps.entity_id = sch.entity_id
            AND dps.scheme_id = sch.scheme_id
            AND order_placed_at >= scheme_active_from 
            AND order_placed_at < scheme_active_to 
    )
    SELECT
        a.entity_id,
        a.test_name,
        a.created_date_utc,
        a.variant,
        a.scheme_id,
        a.dps_travel_time,
        -- Vendor data and information on the delivery
        a.vendor_id,
        a.entity_cluster,
        a.area_cluster,

        -- Business KPIs (These are the components of profit)
        a.platform_order_code,
        a.dps_delivery_fee_local,
        a.dps_travel_time_fee_local,
        a.dps_surge_fee_local,
        a.delivery_fee_local,
        a.gfv_local gfv_local,
        a.gmv_local gmv_local,
        CAST(a.tt_tier AS string) tt_tier
    FROM dps_order_with_pm AS a
 """.format(test_start_date=test_start_date, test_end_date=test_end_date, test_names=", ".join(test_names), entity_id=entity_id, scheme_ids=', '.join(scheme_ids))

# Execute the orders query
df_test_data = client.query(query=test_query).result().to_dataframe(progress_bar_type="tqdm", bqstorage_client=bqstorage_client)

# Change the data types of numeric columns to FLOAT
numeric_cols = ["dps_delivery_fee_local", "dps_travel_time_fee_local", "dps_surge_fee_local", "delivery_fee_local", "gfv_local", "gmv_local"]
df_test_data[numeric_cols] = df_test_data[numeric_cols].applymap(lambda x: float(x))

# Change scheme_id to INT
df_test_data["scheme_id"] = df_test_data["scheme_id"].apply(lambda x: int(x))

# Add the DF/AFV column
df_test_data["df_afv"] = (df_test_data["delivery_fee_local"] / df_test_data["gfv_local"]) * 100

df_test_data.head()

Downloading: 100%|██████████|


,entity_id,test_name,created_date_utc,variant,scheme_id,dps_travel_time,vendor_id,entity_cluster,area_cluster,platform_order_code,dps_delivery_fee_local,dps_travel_time_fee_local,dps_surge_fee_local,delivery_fee_local,gfv_local,gmv_local,tt_tier,df_afv
0,PY_CL,CL_20221201_R_B0_O_ElasticityCLRestaurants,2022-12-30,Control,635,6.131271121,335169,"High basket, high distance",Central cluster,704208299,1190.0,890.0,300.0,1190.0,12730.0,14430.0,3,9.347997
1,PY_CL,CL_20221201_R_B0_O_ElasticityCLRestaurants,2023-01-01,Variation1,634,7.224763784,269542,"Low basket, high distance","Low basket, high distance",705125366,1590.0,1290.0,300.0,1590.0,13000.0,11210.0,4,12.230769
2,PY_CL,CL_20221201_R_B0_O_ElasticityCLRestaurants,2022-12-26,Variation1,634,4.523073791,181704,"Low basket, high distance","Low basket, high distance",701218767,1590.0,1090.0,500.0,2090.0,12600.0,14690.0,3,16.587302
3,PY_CL,CL_20221201_R_B0_O_ElasticityCLRestaurants,2022-12-24,Control,635,2.540174675,141610,"High basket, low distance","Low basket, low distance",700538538,590.0,490.0,100.0,1200.0,15250.0,16450.0,1,7.868852
4,PY_CL,CL_20221201_R_B0_O_ElasticityCLRestaurants,2022-12-31,Variation1,634,8.569887177,232301,"Low basket, high distance","Low basket, high distance",704895627,2090.0,1290.0,800.0,2090.0,3900.0,6340.0,4,53.589744


### Define a function to run the bootstrapping analysis sequentially

In [111]:
# Define a function to run the bootstrap analysis sequentially
def run_bootstrap_sequential(cluster_type):
    vendor_labels = df_test_data["entity_cluster"].unique().tolist()
    price_tiers = [int(tt) for tt in df_test_data["tt_tier"].unique().tolist()]
    price_tiers.sort()

    sim_results = []
    sim_counter = 1
    for i in test_names:
        for j in vendor_labels:
            for k in price_tiers:
                # Filter for the right subset of data
                df_sub_test = df_test_data[(df_test_data["test_name"] == i) & (df_test_data[cluster_type] == j) & (df_test_data["tt_tier"] == k)]

                # Elasticity is calculated by pct difference in orders / pct difference in average DF
                for l in range(1, num_bootstrap_samples + 1):
                    print(f"Iteration {sim_counter}. Sampling with the following parameters --> test_name: {i}, vendor_group_label: {j}, price_tier:{k}, sample_num: {l}")
                    df_ctl_sample = df_sub_test[df_sub_test["variant"] == "Control"].sample(frac=1, replace=True)
                    df_var_sample = df_sub_test[df_sub_test["variant"] == "Variation1"].sample(frac=1, replace=True)
                    
                    num_orders_ctl = df_ctl_sample["platform_order_code"].nunique()
                    num_orders_var = df_var_sample["platform_order_code"].nunique()

                    try:
                        avg_df_ctl = df_ctl_sample["delivery_fee_local"].sum() / num_orders_ctl
                        avg_df_var = df_var_sample["delivery_fee_local"].sum() / num_orders_var

                        avg_tt_fee_ctl = df_ctl_sample["dps_travel_time_fee_local"].sum() / num_orders_ctl
                        avg_tt_fee_var = df_var_sample["dps_travel_time_fee_local"].sum() / num_orders_var

                        pct_diff_orders = float((num_orders_var - num_orders_ctl) / num_orders_ctl)
                        pct_diff_avg_df = float((avg_df_var - avg_df_ctl) / avg_df_ctl)
                        pct_diff_tt_fee = float((avg_tt_fee_var - avg_tt_fee_ctl) / avg_tt_fee_ctl)

                        elasticity = pct_diff_orders / pct_diff_avg_df
                        avg_df_afv_var = float(df_var_sample["delivery_fee_local"].sum() / df_var_sample["gfv_local"].sum())

                        output_dict = {
                            "test_name": i,
                            "vendor_group_label": j,
                            "price_tier": k,
                            "sample_num": l,
                            "avg_df_ctl": avg_df_ctl,
                            "avg_df_var": avg_df_var,
                            "avg_tt_fee_ctl": avg_tt_fee_ctl,
                            "avg_tt_fee_var": avg_tt_fee_var,
                            "orders_tier_ctl": num_orders_ctl,
                            "orders_tier_var": num_orders_var,
                            "total_orders_tier": num_orders_ctl + num_orders_var,
                            "elasticity": elasticity,
                            "pct_diff_orders": pct_diff_orders,
                            "pct_diff_avg_df": pct_diff_avg_df,
                            "pct_diff_tt_fee": pct_diff_tt_fee,
                            "df_afv": avg_df_afv_var
                        }

                        sim_results.append(output_dict)
                            
                        # Increment the sim counter
                        sim_counter += 1
                    except Exception as err:
                        print(err)
                        break

    df_sim_results = pd.DataFrame(sim_results)
    return df_sim_results

### Define a function to run the bootstrapping analysis in parallel

In [121]:
# Define a function to run the bootstrap analysis sequentially
def run_bootstrap_parallel(test_name, cluster_type, vendor_label, price_tier, sample_num, sim_counter):
    # Filter for the right subset of data
    df_sub_test = df_test_data[(df_test_data["test_name"] == test_name) & (df_test_data[cluster_type] == vendor_label) & (df_test_data["tt_tier"] == price_tier)]

    # Elasticity is calculated by pct difference in orders / pct difference in average DF
    print(f"Iteration {sim_counter}. Sampling with the following parameters --> test_name: {test_name}, vendor_group_label: {vendor_label}, price_tier:{price_tier}, sample_num: {sample_num}")
    df_ctl_sample = df_sub_test[df_sub_test["variant"] == "Control"].sample(frac=1, replace=True)
    df_var_sample = df_sub_test[df_sub_test["variant"] == "Variation1"].sample(frac=1, replace=True)
    
    num_orders_ctl = df_ctl_sample["platform_order_code"].count()
    num_orders_var = df_var_sample["platform_order_code"].count()

    avg_df_ctl = df_ctl_sample["delivery_fee_local"].sum() / num_orders_ctl
    avg_df_var = df_var_sample["delivery_fee_local"].sum() / num_orders_var

    avg_tt_fee_ctl = df_ctl_sample["dps_travel_time_fee_local"].sum() / num_orders_ctl
    avg_tt_fee_var = df_var_sample["dps_travel_time_fee_local"].sum() / num_orders_var

    pct_diff_orders = float((num_orders_var - num_orders_ctl) / num_orders_ctl)
    pct_diff_avg_df = float((avg_df_var - avg_df_ctl) / avg_df_ctl)
    pct_diff_tt_fee = float((avg_tt_fee_var - avg_tt_fee_ctl) / avg_tt_fee_ctl)

    elasticity = pct_diff_orders / pct_diff_avg_df
    avg_df_afv_var = float(df_var_sample["delivery_fee_local"].sum() / df_var_sample["gfv_local"].sum())

    output_dict = {
        "test_name": test_name,
        "cluster_type": cluster_type,
        "vendor_group_label": vendor_label,
        "price_tier": price_tier,
        "sample_num": sample_num,
        "avg_df_ctl": avg_df_ctl,
        "avg_df_var": avg_df_var,
        "avg_tt_fee_ctl": avg_tt_fee_ctl,
        "avg_tt_fee_var": avg_tt_fee_var,
        "orders_tier_ctl": num_orders_ctl,
        "orders_tier_var": num_orders_var,
        "total_orders_tier": num_orders_ctl + num_orders_var,
        "elasticity": elasticity,
        "pct_diff_orders": pct_diff_orders,
        "pct_diff_avg_df": pct_diff_avg_df,
        "pct_diff_tt_fee": pct_diff_tt_fee,
        "df_afv": avg_df_afv_var
    }
    df_sim_results = pd.DataFrame([output_dict])

    return df_sim_results

In [122]:
# Define the looping parameters that will be used to define the arguments to be passed to the concurrent.futures method
test_names = df_test_data["test_name"].unique().tolist()
clusters = ["area_cluster", "entity_cluster"]
vendor_labels = df_test_data["entity_cluster"].unique().tolist()
price_tiers = [tt for tt in df_test_data["tt_tier"].unique().tolist()]
price_tiers.sort()
sample_num = np.arange(1, num_bootstrap_samples + 1)

# Define the arguments that will be used in the concurrent.futures method
args_names = []
sim_counter = 1
for i in test_names:
    for j in clusters:
        for k in vendor_labels:
            for l in price_tiers:
                for s in sample_num:
                    arg_iter = tuple([i, j, k, l, s, sim_counter])
                    args_names.append(arg_iter)
                    sim_counter += 1

# Print a message showing the total number of simulations
print(f"The total number of simulations is: {len(args_names)}")

The total number of simulations is: 50000


In [ ]:
# Use concurrent.futures to call the "run_bootstrap" on the list of arguments defined above
# This returns a futures object containing the return value of the function, encluding any exceptions
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = {executor.submit(run_bootstrap_parallel, arg[0], arg[1], arg[2], arg[3], arg[4], arg[5]): arg for arg in args_names}

Iteration 4. Sampling with the following parameters --> test_name: CL_20221201_R_B0_O_ElasticityCLRestaurants, vendor_group_label: High basket, high distance, price_tier:1, sample_num: 4
Iteration 2. Sampling with the following parameters --> test_name: CL_20221201_R_B0_O_ElasticityCLRestaurants, vendor_group_label: High basket, high distance, price_tier:1, sample_num: 2
Iteration 1. Sampling with the following parameters --> test_name: CL_20221201_R_B0_O_ElasticityCLRestaurants, vendor_group_label: High basket, high distance, price_tier:1, sample_num: 1
Iteration 3. Sampling with the following parameters --> test_name: CL_20221201_R_B0_O_ElasticityCLRestaurants, vendor_group_label: High basket, high distance, price_tier:1, sample_num: 3
Iteration 7. Sampling with the following parameters --> test_name: CL_20221201_R_B0_O_ElasticityCLRestaurants, vendor_group_label: High basket, high distance, price_tier:1, sample_num: 7
Iteration 6. Sampling with the following parameters --> test_name

Iteration 14731. Sampling with the following parameters --> test_name: CL_20221201_R_B0_O_ElasticityCLRestaurants, vendor_group_label: High basket, low distance, price_tier:5, sample_num: 731

Iteration 14732. Sampling with the following parameters --> test_name: CL_20221201_R_B0_O_ElasticityCLRestaurants, vendor_group_label: High basket, low distance, price_tier:5, sample_num: 732
Iteration 14735. Sampling with the following parameters --> test_name: CL_20221201_R_B0_O_ElasticityCLRestaurants, vendor_group_label: High basket, low distance, price_tier:5, sample_num: 735
Iteration 14736. Sampling with the following parameters --> test_name: CL_20221201_R_B0_O_ElasticityCLRestaurants, vendor_group_label: High basket, low distance, price_tier:5, sample_num: 736
Iteration 14737. Sampling with the following parameters --> test_name: CL_20221201_R_B0_O_ElasticityCLRestaurants, vendor_group_label: High basket, low distance, price_tier:5, sample_num: 737
Iteration 14738. Sampling with the foll

: 

In [119]:
# Process each result as it is available and append it to df_sim_results. The elements will be added in the order the task is completed, not the order in which they are listed
df_sim_results = []
df_failed = []
for future in concurrent.futures.as_completed(futures):
    try:
        df_sim_results.append(future.result())
    except Exception as err:
        df_failed.append(futures[future][0])

# Convert df_sim_results from a list to a pandas data frame
df_sim_results = pd.concat(df_sim_results)

# Reset the index
df_sim_results = df_sim_results.reset_index(drop=True)

# Display the head of df_sim_results
df_sim_results.head()

,test_name,cluster_type,vendor_group_label,price_tier,sample_num,avg_df_ctl,avg_df_var,avg_tt_fee_ctl,avg_tt_fee_var,orders_tier_ctl,orders_tier_var,total_orders_tier,elasticity,pct_diff_orders,pct_diff_avg_df,pct_diff_tt_fee,df_afv
0,CL_20221201_R_B0_O_ElasticityCLRestaurants,entity_cluster,"High basket, high distance",5,197,2088.595436,2415.982719,2028.550559,2341.618830,10473,10069,20542,-0.246095,-0.038575,0.156750,0.154331,0.075364
1,CL_20221201_R_B0_O_ElasticityCLRestaurants,entity_cluster,"High basket, high distance",1,209,843.912469,1159.084437,775.070741,1096.195373,5301,5057,10358,-0.123249,-0.046029,0.373465,0.414317,0.039096
2,CL_20221201_R_B0_O_ElasticityCLRestaurants,area_cluster,Central cluster,1,155,837.696649,1159.024326,781.320988,1101.717711,11340,11428,22768,0.020231,0.007760,0.383585,0.410071,0.052998
3,CL_20221201_R_B0_O_ElasticityCLRestaurants,entity_cluster,"High basket, high distance",5,196,2117.700783,2424.636318,2053.253117,2358.955478,10347,9995,20342,-0.234718,-0.034020,0.144938,0.148887,0.074456
4,CL_20221201_R_B0_O_ElasticityCLRestaurants,entity_cluster,"High basket, high distance",1,208,841.232102,1164.744584,774.048606,1101.860465,5308,5031,10339,-0.135698,-0.052185,0.384570,0.423503,0.038710


In [120]:
# Upload the data frame to BigQuery
job_config = bigquery.LoadJobConfig()
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
client.load_table_from_dataframe(
    dataframe=df_sim_results,
    destination="logistics-data-storage-staging.long_term_pricing.vendor_cluster_tier_level_analysis",
    job_config=job_config
).result()

LoadJob<project=logistics-customer-staging, location=US, id=d0a2f8c7-6622-4f09-a1ac-9ab2452e3612>

In [ ]:
# Calculate the mean elasticity of All the clusters per price tier.
def percentile_left(x):
    return x.quantile(left_threshold)

def percentile_right(x):
    return x.quantile(right_threshold)

list_of_agg_functions = ["mean", percentile_left, percentile_right]
agg_func_selection = {"elasticity": list_of_agg_functions, "df_afv": list_of_agg_functions, "pct_diff_orders": list_of_agg_functions, "pct_diff_avg_df": list_of_agg_functions}
df_stats = df_sim_results.groupby(["test_name","price_tier", "vendor_group_label"])[["elasticity", "pct_diff_orders", "pct_diff_avg_df", "df_afv"]].agg(agg_func_selection).reset_index()
df_stats

In [ ]:
min_elas = -4.0
max_elas = 4.0
df_sim_filtered = df_sim_results[(df_sim_results['elasticity'] >= min_elas) & (df_sim_results['elasticity'] <= max_elas)]
df_sim_filtered

In [ ]:
# Filter dataset by relevant columns
df_filtered = df_sim_filtered[['price_tier', 'vendor_group_label', 'elasticity']]

# Generate a subplot for each price tier
fig, axs = plt.subplots(len(df_filtered['price_tier'].unique()), figsize=(10, 6*len(df_filtered['price_tier'].unique())))

# Iterate over price tier values and create the histograms
for i, price_tier in enumerate(df_filtered['price_tier'].unique()):
    ax = axs[i] if len(df_filtered['price_tier'].unique()) > 1 else axs
    filtered_data = df_filtered[df_filtered['price_tier'] == price_tier]
    for vendor_group_label in filtered_data['vendor_group_label'].unique():
        vendor_data = filtered_data[filtered_data['vendor_group_label'] == vendor_group_label]
        ax.hist(vendor_data['elasticity'], bins=20, alpha=0.5, label=vendor_group_label)
    ax.set_title(f'Price Tier: {price_tier}')
    ax.set_xlabel('Elasticity')
    ax.set_ylabel('Frequency')
    ax.legend()

# Adjust the space between the subplots
plt.tight_layout()

# Show plot
plt.show()